In [2]:
import matplotlib.pyplot as plt
import datetime
import torch
import torch.nn as nn
import torch.nn.functional as F
import copy
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torchvision.models as models

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [3]:
class Net(nn.Module):
        def __init__(self):
                super(Net, self).__init__()
                self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
                self.conv2 = nn.Conv2d(32, 32, kernel_size=5)
                self.conv3 = nn.Conv2d(32,64, kernel_size=5)
                self.fc1 = nn.Linear(3*3*64, 256)
                self.fc2 = nn.Linear(256, 10)
                
        def forward(self, x):
                x = F.relu(self.conv1(x))
                x = F.relu(F.max_pool2d(self.conv2(x), 2))
                x = F.dropout(x, p=0.5, training=self.training)
                x = F.relu(F.max_pool2d(self.conv3(x),2))
                x = F.dropout(x, p=0.5, training=self.training)
                x = x.view(-1,3*3*64 )
                x = F.relu(self.fc1(x))
                x = F.dropout(x, training=self.training)
                x = self.fc2(x)
                return F.log_softmax(x, dim=1)

model = Net()

In [4]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs):

    train_losses = []
    valid_losses = []
    best_model_loss = 5
    
    for epoch in range(num_epochs):

        # Training
        for i, (data, labels) in enumerate(train_loader):
      
            prediction = model.forward(data)

            train_loss = criterion(prediction, labels)

            train_loss.backward()

            optimizer.step()

            optimizer.zero_grad()
        print(f'\rEpoch {epoch+1}, batch {i+1}/{len(train_loader)} - Loss: {train_loss}')

        train_losses.append(train_loss)
        writer.add_scalar("Loss/train ADAM", train_loss, epoch)

        # Validation
        for batch_nr, (data, labels) in enumerate(val_loader):
            prediction = model.forward(data)
            loss_val = criterion(prediction, labels)
            valid_losses.append(loss_val)
        print(f"loss validation: {loss_val}")
        #print(f"loss validation: {loss_val}","\n")

        if valid_losses[-1] < best_model_loss:
            print(f"\t > Found a better model, {best_model_loss} -> {valid_losses[-1]}")
            best_model = copy.deepcopy(model)
            best_model_loss = valid_losses[-1]

        writer.add_scalar("Loss/validation ADAM", loss_val, epoch)

    print(f"\nBest model loss: {best_model_loss}")
    return best_model, train_losses, valid_losses

In [5]:
def get_accuracy(network, loader):
    
    with torch.no_grad():
        correct = 0
        total = 0
        y_pred = []
        y_true = []

        for x, (data, labels) in enumerate(loader):

            prediction = network.forward(data)

            for i in range(len(data)):

                y_true.append(labels[i].item())
                y_pred.append(torch.argmax(prediction[i]).item())
                if y_true[i] == y_pred[i]:
                    correct += 1        
    
            total += float(len(data))
    
        score = correct/total

        accuracy = score

        return accuracy

In [6]:
LEARNING_RATE = 0.0001
EPOCHS = 4
BATCH_SIZE = 1000

transform = transforms.Compose([transforms.ToTensor()])

trainset = torchvision.datasets.MNIST(root='./data', train=True,download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data', train=False,download=True, transform=transform)

validset, trainset = torch.utils.data.random_split(trainset, [10000, 50000])

trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,shuffle=True)
validloader = torch.utils.data.DataLoader(validset, batch_size=BATCH_SIZE,shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,shuffle=False)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

# Train the model
trained_model, train_loss, valid_loss = train_model(model, criterion, optimizer, trainloader, validloader, EPOCHS)

# Test the model
test_acc = get_accuracy(trained_model, testloader)
print(f"Model Accuracy (MNIST): {test_acc*100}%")
writer.flush()

Epoch 1, batch 50/50 - Loss: 2.022226095199585
loss validation: 1.985370397567749
	 > Found a better model, 5 -> 1.985370397567749
Epoch 2, batch 50/50 - Loss: 0.9370052814483643
loss validation: 0.9804132580757141
	 > Found a better model, 1.985370397567749 -> 0.9804132580757141
Epoch 3, batch 50/50 - Loss: 0.6620936989784241
loss validation: 0.6375173926353455
	 > Found a better model, 0.9804132580757141 -> 0.6375173926353455
Epoch 4, batch 50/50 - Loss: 0.5638431906700134
loss validation: 0.49907007813453674
	 > Found a better model, 0.6375173926353455 -> 0.49907007813453674

Best model loss: 0.49907007813453674
Model Accuracy (MNIST): 83.6%


**SVHN**

In [7]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),transforms.Grayscale(num_output_channels=1),transforms.Resize(28)])

dataset = torchvision.datasets.SVHN(root='./data',download=True,transform=transform)

testset, validset, trainset = torch.utils.data.random_split(dataset, [10000,12000,51257])

trainloader_svhn = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,shuffle=True)
validloader_svhn = torch.utils.data.DataLoader(validset, batch_size=BATCH_SIZE,shuffle=True)
testloader_svhn = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,shuffle=False)

# Test the model
test_acc = get_accuracy(trained_model, testloader_svhn)
print(f"Model Accuracy (SVHN): {test_acc*100}%")
writer.flush()

Using downloaded and verified file: ./data\train_32x32.mat
Model Accuracy (SVHN): 13.3%


**Transfer Learning**

In [8]:
model_finetune = trained_model


LEARNING_RATE = 0.01
EPOCHS = 5
BATCH_SIZE = 1000

# Define our loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = LEARNING_RATE)

# Train the model
trained_model_finetuned, train_loss, valid_loss = train_model(model_finetune, criterion, optimizer, trainloader_svhn, validloader_svhn, EPOCHS)

# Test the model
test_acc = get_accuracy(trained_model_finetuned, testloader_svhn)
print(f"Model Accuracy (Transfer Leraning): {test_acc*100}%")
writer.flush()

Epoch 1, batch 52/52 - Loss: 4.225551128387451
loss validation: 4.543282508850098
	 > Found a better model, 5 -> 4.543282508850098
Epoch 2, batch 52/52 - Loss: 3.7912509441375732
loss validation: 4.601983547210693
Epoch 3, batch 52/52 - Loss: 4.6349382400512695
loss validation: 4.298398494720459
	 > Found a better model, 4.543282508850098 -> 4.298398494720459
Epoch 4, batch 52/52 - Loss: 4.117882251739502
loss validation: 4.166539192199707
	 > Found a better model, 4.298398494720459 -> 4.166539192199707
Epoch 5, batch 52/52 - Loss: 4.53678035736084
loss validation: 4.564538955688477

Best model loss: 4.166539192199707
Model Accuracy (Transfer Leraning): 14.7%
